In [ ]:
#Installing required Packages
!pip install selenium
!pip install webdriver_manager


In [1]:
#Importing packages
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By

import re
import time
import os

In [2]:
#Setting chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\anvin\.wdm\drivers\chromedriver\win32\97.0.4692.71]
C:\Users\anvin\AppData\Local\Temp/ipykernel_14956/2433240595.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
C:\Users\anvin\AppData\Local\Temp/ipykernel_14956/2433240595.py:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)


In [3]:
wait = WebDriverWait(browser, 20)

In [4]:
#Invoking  browser with url
#smartphone_url='https://www.amazon.in/s?rh=n%3A1805560031&fs=true&ref=lp_1805560031_sar'
smartphone_url='https://www.amazon.in/s?i=electronics&bbn=1805560031&rh=n%3A1805560031%2Cp_89%3ATecno&dc&fs=true&qid=1642520011&rnid=3837712031&ref=sr_nr_p_89_6'


In [5]:
def get_lastpage_number():
    try:
        xpath_pagination = '//span[@class="s-pagination-strip"]'
        last_page_obj = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_pagination)))
        last_page_text = last_page_obj.text
        last_page_match_obj = re.search(r'\.\s*(\d+)\s*Next', last_page_text)
        if last_page_match_obj != None:
            last_page = last_page_match_obj.group(1).strip()
        else:
            last_page_match_obj = re.search(r'(\d)\s*Next', last_page_text)
            last_page = last_page_match_obj.group(1).strip() if last_page_match_obj != None else ''
    except Exception as e:
        try:
            xpath_pagination = '//ul[@class="a-pagination"]'
            last_page_obj = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_pagination)))
            last_page_text = last_page_obj.text
            last_page_match_obj = re.search(r'(\d+)\nNext', last_page_text)
            last_page = last_page_match_obj.group(1).strip() if last_page_match_obj != None else ''        
        except Exception as e:
            print(e)
            last_page=''
    return last_page

In [6]:
def get_product_links():
    browser.get(smartphone_url)
    last_page_number = get_lastpage_number()
    print("Number of pages to scrape : ", last_page_number)
    xpath_next_btn_pagination_1 = '//li[@class="a-last"]'
    xpath_next_btn_pagination_2 = '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'

    last_page_number = int(last_page_number) + 1
    #looping through pagination links
    xpath_product_link = '//a[@class="a-link-normal s-no-outline"]'
    product_links = []
    for i in range(1, last_page_number):
        if i == 1:
            browser.get(smartphone_url)

        links = wait.until(ec.visibility_of_all_elements_located((By.XPATH, xpath_product_link)))
        for link in links:
            product_links.append(link.get_attribute('href'))

        if i == last_page_number-1:
            break
        time.sleep(3)
        try:
            next_btn = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_next_btn_pagination_1)))
            next_btn.click()
        except Exception as e:
            next_btn = wait.until(ec.visibility_of_element_located((By.XPATH, xpath_next_btn_pagination_2)))
            next_btn.click()
    print('Number of product links found : ',len(product_links))   
    return product_links

In [21]:
def get_product_details(product_url):
    print(product_url)
    browser.get(product_url)
    prod_info = wait.until(ec.visibility_of_element_located((By.ID,'centerCol')))
    prod_info = prod_info.text
    prod_info = os.linesep.join([s for s in prod_info.splitlines() if s])
    
    prod_tech_info = wait.until(ec.visibility_of_element_located((By.ID,'productDetails_techSpec_section_1')))
    prod_tech_info = prod_tech_info.text
    
    prod_additional_info = wait.until(ec.visibility_of_element_located((By.ID,'productDetails_detailBullets_sections1')))
    prod_additional_info = prod_additional_info.text
    
    prod_buy_info = wait.until(ec.visibility_of_element_located((By.ID,'desktop_qualifiedBuyBox')))
    prod_buy_info = prod_buy_info.text
    
    prod_data = prod_info +'\n\n'+ prod_tech_info +'\n\n'+ prod_additional_info +'\n\n'+ prod_buy_info
    
    #print(prod_data)
    return prod_data
    

In [61]:
def extract_pruduct_info(prod_data):
    prod_mrp_price_obj = re.search(r'm\.r\.p\.\:\s*\n(.*)', prod_data, re.IGNORECASE)
    prod_mrp_price = prod_mrp_price_obj.group(1) if prod_mrp_price_obj != None else ''
    print('prod_mrp_price :',prod_mrp_price)
    
    deal_of_the_day_obj = re.search(r'deal\s+of\s+the\s+day\:\s*\n(.*)', prod_data, re.IGNORECASE)
    deal_of_the_day = deal_of_the_day_obj.group(1) if deal_of_the_day_obj != None else ''
    print('deal_of_the_day :',deal_of_the_day)
    
    deal_end_time_obj = re.search(r'ends\s+in\s+(.*)', prod_data, re.IGNORECASE)
    deal_end_time = deal_end_time_obj.group(1) if deal_end_time_obj != None else ''
    print('deal_end_time :',deal_end_time)
    
    your_savings_obj = re.search(r'you\s+save\:\s*\n(.*)', prod_data, re.IGNORECASE)
    your_savings = your_savings_obj.group(1) if your_savings_obj != None else ''
    print('your_savings :',your_savings)
    
    emi_available_obj = re.search(r'emi\s+starts\s+at', prod_data, re.IGNORECASE)
    emi_available = 'yes' if emi_available_obj != None else ''
    print('emi_available :',emi_available)
    
    nocost_emi_available_obj = re.search(r'no\s+cost\s+emi\s+available', prod_data, re.IGNORECASE)
    nocost_emi_available = 'yes' if nocost_emi_available_obj != None else ''
    print('nocost_emi_available :',nocost_emi_available)
    
    product_model_obj = re.search(r'model\s+name\s+(.*)', prod_data, re.IGNORECASE)
    product_model = product_model_obj.group(1) if product_model_obj != None else ''
    print('product_model :',product_model)
    
    wireless_carrier_obj = re.search(r'wireless\s+carrier\s+(.*)', prod_data, re.IGNORECASE)
    wireless_carrier = wireless_carrier_obj.group(1) if wireless_carrier_obj != None else ''
    print('wireless_carrier :',wireless_carrier)
    
    product_brand_obj = re.search(r'brand\s+(.*)', prod_data, re.IGNORECASE)
    product_brand = product_brand_obj.group(1) if product_brand_obj != None else ''
    print('product_brand :',product_brand)
    
    product_memory_storage_obj = re.search(r'memory\s+storage\s+capacity\s+(.*)', prod_data, re.IGNORECASE)
    product_memory_storage = product_memory_storage_obj.group(1) if product_memory_storage_obj != None else ''
    print('product_memory_storage :',product_memory_storage)
    
    product_os_obj = re.search(r'\nos\s+(.*)', prod_data, re.IGNORECASE)
    product_os = product_os_obj.group(1) if product_os_obj != None else ''
    print('product_os :',product_os)
    
    product_ram_obj = re.search(r'\nram\s+(.*)', prod_data, re.IGNORECASE)
    product_ram = product_ram_obj.group(1) if product_ram_obj != None else ''
    print('product_ram :',product_ram)
    
    product_colour_obj = re.search(r'\ncolour\s+(.*)', prod_data, re.IGNORECASE)
    product_colour = product_colour_obj.group(1) if product_colour_obj != None else ''
    print('product_colour :',product_colour)

    product_manufacturer_obj = re.search(r'\nmanufacturer\s+(.*)', prod_data, re.IGNORECASE)
    product_manufacturer = product_manufacturer_obj.group(1) if product_manufacturer_obj != None else ''
    print('product_manufacturer :',product_manufacturer)
    
    product_country_of_origin_obj = re.search(r'\ncountry\s+of\s+origin\s+(.*)', prod_data, re.IGNORECASE)
    product_country_of_origin = product_country_of_origin_obj.group(1) if product_country_of_origin_obj != None else ''
    print('product_country_of_origin :',product_country_of_origin)











In [62]:
product_links = get_product_links()

In [ ]:
for p_link in product_links:
    prod_data = get_product_details(p_link)
    extract_pruduct_info(prod_data)
    break # for checking only single product

https://www.amazon.in/Tecno-5000mAh-Camera-Splash-Resistant/dp/B09PRDHRKV/ref=sr_1_1?qid=1643989191&refinements=p_89%3ATecno&rnid=3837712031&s=electronics&sr=1-1
